# 举例1: 大模型分析工具的调用

In [2]:
import os
import dotenv
from langchain_community.tools import MoveFileTool
from langchain_core.messages import HumanMessage
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai import ChatOpenAI


# 1. 获取大模型

dotenv.load_dotenv()

chat_model = ChatOpenAI(
    model_name = os.getenv("MODEL"),
    api_key = os.getenv("ARK_API_KEY"),
    base_url = os.getenv("BASE_URL"),
    temperature = 0
)


# 2. 获取工具的列表
tools = [MoveFileTool()]

# 3. 因为大模型invoke调用时，需要传入函数的列表，所以需要将工具转换为函数
functions = [convert_to_openai_function(t) for t in tools]


# 4. 获取消息列表
messages = [HumanMessage(content="将文件a移动到桌面")]

# 5. 调用大模型(传入消息列表、工具的列表)
response =  chat_model.invoke(
    input=messages,
    functions = functions,
)

print(response)



content='' additional_kwargs={'function_call': {'arguments': '{"source_path":"a","destination_path":"/Users/YourUsername/Desktop/a"}', 'name': 'move_file'}, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 76, 'total_tokens': 103, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-CRGAwTFld2X2TULkeVsFK1rx2YPTp', 'service_tier': None, 'finish_reason': 'function_call', 'logprobs': None} id='run--fbb1283b-1389-4e2f-9e2f-6b8be8f69d9d-0' usage_metadata={'input_tokens': 76, 'output_tokens': 27, 'total_tokens': 103, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


# 举例2: 如何调用具体大模型分析出来的工具

## 说明:
1. 大模型与Agent的核心区别:
2. 针对于大模型: 仅能分析出要调用的工具,但是此工具(或函数)不能真正的执行
针对于Agent: 除了分析出要调用的工具之外,还可以执行具体的工具(或函数)

In [5]:
import json
import os
import dotenv
from langchain_community.tools import MoveFileTool
from langchain_core.messages import HumanMessage
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai import ChatOpenAI


# 1. 获取大模型

dotenv.load_dotenv()

chat_model = ChatOpenAI(
    model_name = os.getenv("MODEL"),
    api_key = os.getenv("ARK_API_KEY"),
    base_url = os.getenv("BASE_URL"),
    temperature = 0
)


# 2. 获取工具的列表
tools = [MoveFileTool()]

# 3. 因为大模型invoke调用时，需要传入函数的列表，所以需要将工具转换为函数
functions = [convert_to_openai_function(t) for t in tools]


# 4. 获取消息列表
messages = [HumanMessage(content="随便生成一个txt文件移动到当前目录")]

# 5. 调用大模型(传入消息列表、工具的列表)
response =  chat_model.invoke(
    input=messages,
    functions = functions,
)

# print(response)
# 步骤1:分析下要调用哪个工具或函数
if "function_call" in response.additional_kwargs:
    tool_name =  response.additional_kwargs["function_call"]["name"]
    tool_args = json.loads(response.additional_kwargs["function_call"]["arguments"])
    print(f"调用工具:{tool_name}\n 参数:{tool_args}")
else:
    print(f"模型回复:{response.content}")
    
# 步骤2: 调用对应的工具
if "move_file" in response.additional_kwargs["function_call"]["name"]:
    tool = MoveFileTool()
    result = tool.run(tool_args)
    print("工具执行的结果:",result)


调用工具:move_file
 参数:{'source_path': '/tmp/sample.txt', 'destination_path': './sample.txt'}
工具执行的结果: Error: no such file or directory /tmp/sample.txt
